## USDA food databases (legacy food database)

+ Data are available in csv and json formats. Current json file do not provide desired format.
+ json will be the chosen file format since we'll work with MongoDB for faster API reponses
+ We will however use CSVs as base to re-build relevant json format accross all 3 DBs

---
### Desired format (v1) 

````json

{
    "legacy": [
        {
            "_id" : "r00000000001",
            "type" : 2 , #define food type (1-food, 2-branded , 3-recipes)
            "category": "vegetables", 
            "title": "Banana com aveia e mel",
            "ingredients" : #optional
                {
                    "f0000000001": {
                        "amount" : 100,
                            "unit" : "g",
                            "name" : "Can be food"},
                    "f0000000002": 
                        {"amount" : 50,
                        "unit" : "g",
                        "name": "Could also be a branded product"},
                    "f0000000003": 
                        {"amount" : 10,
                        "unit" : "g",
                        "name" : "Even another recipe- like home made cheese"}                       
                },
            "nutrients" : 
                {
                    "f0000000001": {
                        "amount" : 100,
                         "unit" : "g",
                         "name" : "carbs"},
                    "f0000000002": 
                        {"amount" : 50,
                        "unit" : "kcal",
                        "name" : "fat"},
                    "f0000000003": 
                        {"amount" : 10,
                        "unit" : "g"}                       
                }
}

```

In [20]:
import pandas as pd

In [21]:
#USDA legacy database select files

food = pd.read_csv(r"./legacy/food.csv").drop(columns='data_type')
nutrients = pd.read_csv(r"./legacy/food_nutrient.csv")

nutrient_detail=pd.read_csv(r"./support/nutrient.csv")\
    .rename(columns={'id':'nutrient_id','name':'nutrient_name', 'unit_name':'nutrient_unit'})

categories = pd.read_csv(r"./support/food_category.csv")\
    .rename(columns={'id':'food_category_id', 'code': 'category_code', 'description':'category_description'})

portion = pd.read_csv(r"./legacy/food_portion.csv").rename(columns={'modifier':'portion'})

In [22]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7793 entries, 0 to 7792
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   fdc_id            7793 non-null   int64 
 1   description       7793 non-null   object
 2   food_category_id  7793 non-null   int64 
 3   publication_date  7793 non-null   object
dtypes: int64(2), object(2)
memory usage: 243.7+ KB


In [23]:
pd.to_datetime(food['publication_date'], format='%Y-%m-%d')
food.head(5)

,fdc_id,description,food_category_id,publication_date
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01
3,167515,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01


In [24]:
food = food.merge(nutrients.loc[:,'fdc_id':'amount'], on='fdc_id', how='left')\
    .rename(columns={'amount': 'nutrient_amount'})

food = food.merge(nutrient_detail[['nutrient_id','nutrient_name','nutrient_unit']],\
    on='nutrient_id', how='left',suffixes=('', '_DROP'))\
        .filter(regex='^(?!.*_DROP)')\
            .astype({'nutrient_id':'int32'})

food['nutrient_unit'] = food['nutrient_unit'].fillna('na').apply(lambda x: x.lower())

food = food.merge(categories, on= 'food_category_id', how='left',suffixes=('', '_DROP'))\
    .filter(regex='^(?!.*_DROP)')

food = food.merge(portion[['fdc_id','portion','gram_weight']], on= 'fdc_id', how='left',suffixes=('', '_DROP'))\
    .filter(regex='^(?!.*_DROP)')

In [25]:
food.head(5)

,fdc_id,description,food_category_id,publication_date,nutrient_id,nutrient_amount,nutrient_name,nutrient_unit,category_code,category_description,portion,gram_weight
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1003,5.88,Protein,g,1800,Baked Products,serving,34.0
1,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1007,3.50,Ash,g,1800,Baked Products,serving,34.0
2,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1062,1286.00,Energy,kj,1800,Baked Products,serving,34.0
3,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1079,1.20,"Fiber, total dietary",g,1800,Baked Products,serving,34.0
4,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1089,2.12,"Iron, Fe",mg,1800,Baked Products,serving,34.0


In [26]:
food.index[-1]

1260772

In [27]:
# keep only items with complete info
mask = (food['gram_weight'].notna() & food['nutrient_id'].notna())
food = food[mask].astype({
        'nutrient_id':'int', 
        'food_category_id':'int',
        'category_code':'int'})

food.head(5)

,fdc_id,description,food_category_id,publication_date,nutrient_id,nutrient_amount,nutrient_name,nutrient_unit,category_code,category_description,portion,gram_weight
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1003,5.88,Protein,g,1800,Baked Products,serving,34.0
1,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1007,3.50,Ash,g,1800,Baked Products,serving,34.0
2,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1062,1286.00,Energy,kj,1800,Baked Products,serving,34.0
3,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1079,1.20,"Fiber, total dietary",g,1800,Baked Products,serving,34.0
4,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1089,2.12,"Iron, Fe",mg,1800,Baked Products,serving,34.0


In [28]:
# Stack the nutrients per fdc_id (prepare for nested json output)
# make it better? Abstract / Faster
# it always group by the id !

def inlist(df, aggcol:list):

    groupbycol = [col for col in df.columns if col not in aggcol ]
    aggregation = {col:lambda x: list(x) for col in aggcol}
    groupid = [col for col in aggcol if 'id' in col]
    df= df.groupby(groupbycol).agg(aggregation).reset_index()

    # add column name as prefix to each aggregated item
    for col in aggcol:
        if col not in groupid:
            df[col] = df[col].apply(lambda x: [{col : v} for v in x])
    return df

aggcol=['nutrient_id',	'nutrient_amount'	,'nutrient_name',	'nutrient_unit']
grupo = inlist(food, aggcol)

In [29]:
from itertools import chain

def izipit(x, head=False):
    keys = (x.keys().unique())
    if head:
        return dict(zip(*x[keys]))
    else:
        arr =  list(map(list,zip(*x[keys])))
        return [dict(chain.from_iterable(map(dict.items, row)))for row in arr]

cols = ['nutrient_amount',	'nutrient_name','nutrient_unit']
grupo['nutrients']=grupo[cols].apply(izipit, axis=1)
grupo.drop(columns=cols, inplace=True)
grupo['nutrients']=grupo[['nutrient_id', 'nutrients']].apply(izipit,args=(True,), axis=1)
grupo.drop(columns='nutrient_id',inplace=True)

In [30]:
grupo.head(5)

,fdc_id,description,food_category_id,publication_date,category_code,category_description,portion,gram_weight,nutrients
0,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",18,2019-04-01,1800,Baked Products,serving,34.0,"{1003: {'nutrient_amount': 5.88, 'nutrient_nam..."
1,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",18,2019-04-01,1800,Baked Products,serving 1 roll with icing,44.0,"{1003: {'nutrient_amount': 4.34, 'nutrient_nam..."
2,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",18,2019-04-01,1800,Baked Products,serving,28.0,"{1062: {'nutrient_amount': 1577.0, 'nutrient_n..."
3,167515,"George Weston Bakeries, Thomas English Muffins",18,2019-04-01,1800,Baked Products,serving,57.0,"{1105: {'nutrient_amount': 0.0, 'nutrient_name..."
4,167516,"Waffles, buttermilk, frozen, ready-to-heat",18,2019-04-01,1800,Baked Products,"waffle, round",38.0,"{1258: {'nutrient_amount': 1.898, 'nutrient_na..."


In [31]:
grupo = grupo.drop_duplicates(subset=['fdc_id','description'],keep='first')

In [32]:
#export to json 
grupo['source'] ='USDA_legacy'
grupo.to_json('legacy.json',orient="records", indent=2)

#TODO - add tables to sql db for future OLAP enrichment

In [33]:
del food
del nutrient_detail
del nutrients
del categories
del grupo